In [ ]:
import ee
import geemap
import os
import glob
from IPython.display import Image

In [ ]:
ee.Initialize()

[country codes](https://en.wikipedia.org/wiki/List_of_FIPS_country_codes)

In [ ]:
country_code = "NI"
african_bb = ee.Geometry.Polygon([[
    [-19.698368046353494, 38.1446395611524],
    [-19.698368046353494, -36.16300755581617],
    [53.229366328646506, -36.16300755581617],
    [53.229366328646506, 38.1446395611524]
]])
bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7']
date_start = '1985-01-01' 
date_end = '2012-12-31'

In [ ]:
# adm2 = ee.FeatureCollection('FAO/GAUL/2015/level2').filter(ee.Filter.eq('wld_rgn', 'Africa'))
# mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('wld_rgn', 'Africa'))
# land_8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA')

## [Large Scale International Boundary Polygons](https://developers.google.com/earth-engine/datasets/catalog/USDOS_LSIB_SIMPLE_2017?hl=en)

In [ ]:
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
mask = mask.filter(ee.Filter.eq('country_co', country_code))

## [Landsat 5 collection 1, tier 1](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT05_C01_T1_TOA#terms-of-use)

In [ ]:
raw_ic = ee.ImageCollection('LANDSAT/LT05/C01/T1')
raw_ic = raw_ic.filterDate(date_start, date_end)
raw_ic = raw_ic.select(bands)

In [ ]:
# nigeria = raw_ic.filterBounds(mask)  # this gives opposite of what I want

In [ ]:
nigeria = raw_ic.filter('CLOUD_COVER < 20').mean().clip(mask)

In [ ]:
red = nigeria.select('B4')
nir = nigeria.select('B5')
ndvi=(nir.subtract(red)).divide((nir.add(red))).rename('NDVI')

In [ ]:
ndviParams = {'min': -1, 
              'max': 1, 
              'palette': ['blue', 'white', 'green']
             }

In [ ]:
roi = mask.geometry()

In [ ]:
url = ndvi.getThumbUrl({
    'min': -1, 
    'max': 1, 
    'dimensions': 512,
    'region': roi,
    'palette': ['blue', 'white', 'green']
})
Image(url=url, embed=True, format = 'png')